In [1]:
from google.cloud import vision
from google.cloud import storage
from PIL import Image
import numpy as np
from helper import *
import os, sys
import csv  


Crop all images in one batch.

In [2]:
# raw_dir = '../Data/raw/'

# all_files = [raw_dir+i for i in os.listdir(raw_dir) if i[-4:] == '.JPG']
# for i in all_files:
#     CroppedImage(i)

Assess predictions

In [3]:
crop_dir = '../Data/cropped/'

all_files = [crop_dir+i for i in os.listdir(crop_dir) if i[-4:] == '.JPG']

In [4]:
with open('../Data/predictions.csv', 'w') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(['filename', 'prediction', 'corrected'])

    for i in range(len(all_files)):    
        print(f'{i}/{len(all_files)}',end='\r')
        # write the data
        prediction = ReadImage(all_files[i], verbose=False)[0]

        corrected_prediction = FindErrors(prediction)

        writer.writerow([all_files[i], prediction, corrected_prediction])

In [6]:
file = '../Data/cropped/DSC_1219.JPG'
output_string, word_confidences = ReadImage(file)
label = FindErrors(output_string,verbose = True)


Removing special characters from the output string (e.g. '.', '|').
Character M is fine.
Character C is fine.
Character A is fine.
Character - is fine.
Character ✩ has been removed.
Character - is fine.
Character T is fine.
Character R is fine.
Character A is fine.
Character V is fine.
Character - is fine.
Character 0 is fine.
Character 1 is fine.
Character / is fine.
Character 2 is fine.
Character 1 is fine.
Character / is fine.
Character 1 is fine.
Character 3 is fine.

Removing paragraphs without three (3), four (4), five (5) or eight (8) elements.
Removed empty paragraph
Label after removing dead paragraphs: ['MCA', 'TRAV', '01/21/13']

Correct number of paragraphs.

Looking for errors in the title (MCA).
Final title: MCA

Looking for errors in the identity (TRAV).
Character T at index 0 has been replace with 1
Character A at index 2 has been replace with 7
Final identity: 1R7V

Looking for errors in the date (01/21/13).
Final date: 01/21/13

Final label is: MCA-1R7V-01/21/13
